In [ ]:
%mkdir data
%cd data
!unzip /content/drive/MyDrive/dataset/animals.zip -d /content/data/

In [2]:
path = "/content/data/animals"

In [3]:
from glob import glob

pth = sorted(glob("/content/data/animals/*"))
CLASSES = [i.split("/")[-1] for i in pth]
print(CLASSES)

['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'ragno', 'sheep', 'squirrel']


In [4]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

class AnimalsImageFolder(Dataset):

  def __init__(self, root, transform=None):
    self.valid_img_files = []
    self.valid_labels = []
    self.transform = transform

    for cls_idx, cls_name in enumerate(CLASSES):
      all_files = glob(os.path.join(root, cls_name, '*.jpeg'))
      all_labes = [cls_idx] * len(all_files)

      self.valid_img_files.extend(all_files)
      self.valid_labels.extend(all_labes)


  def __len__(self):
    return len(self.valid_img_files)

  def __getitem__(self, idx):
    img = Image.open(self.valid_img_files[idx])

    if self.transform is not None:
      img = self.transform(img)

    label = self.valid_labels[idx]
    label = torch.tensor(label, dtype=torch.long)

    return img, label

# Transform

def get_transform(train):

  transforms = []
  transforms.append(T.Resize((224,224)))
  transforms.append(T.ToTensor())
  transforms.append(T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
  if train:
    transforms.append(T.RandomHorizontalFlip(0.5))

  return T.Compose(transforms)

In [5]:
from torch import nn
from torchvision.models import resnet50

# requires_grad: By default, when we load a pretrained model all of the parameters
# have .requires_grad=True, which is fine if we are training from scratch or finetuning.
# However, if we are feature extracting and only want to compute gradients for the newly
# initialized layer then we want all of the other parameters to not require gradients.

def AnimalResnet(use_pretrained=True, 
               feature_extracting=True, n_class=10, drop_rate=0.2):
  
    model = resnet50(pretrained=use_pretrained)

    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, n_class)

    return model

In [6]:
model = AnimalResnet(use_pretrained=True, feature_extracting=True, n_class=10, drop_rate=0.2)
print(model)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
data = AnimalsImageFolder(path, transform=get_transform(train=True))
data_test = AnimalsImageFolder(path, transform=get_transform(train=False))

torch.manual_seed(1)
indices = torch.randperm(len(data)).tolist()
split_idx = int(0.2 * len(data))

trainset = torch.utils.data.Subset(data, indices[:-split_idx])
valset = torch.utils.data.Subset(data_test, indices[-split_idx:])

print("Total no.", len(data), len(trainset), len(valset))

Total no. 24209 19368 4841


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_lr = 3e-5
epochs = 2
weight_decay = 1e-4
batch_size = 20

In [9]:
from torch.utils.data import DataLoader
from collections import Counter

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=True)

class_weights = False
if class_weights:
    class_counts = dict(Counter(data.targets))
    m = max(class_counts.values())
    for c in class_counts:
        class_counts[c] = m / class_counts[c]
    weights = []
    for k in sorted(class_counts.keys()):
        weights.append(class_counts[k])

    weights = torch.Tensor(weights)
    loss_fn = nn.CrossEntropyLoss(weight=weights.to(device))
else:
    loss_fn = torch.nn.CrossEntropyLoss()




optimizer = torch.optim.Adam(model.parameters(), max_lr, weight_decay=weight_decay)
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))


In [10]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np

model = model.to(device)

val_acc = 0.0 
for epoch in range(epochs):
  model.train()
  losses = 0
  with tqdm(train_loader, unit="batch") as tepoch:
    for data, target in tepoch:

      optimizer.zero_grad()
      pred = model(data.to(device))
      
      loss = loss_fn(pred, target.to(device))
      loss.backward()
      optimizer.step()
      sched.step()

      losses += loss.item()

  train_loss = losses/len(train_loader)
  print('Train Epoch {} Training Loss {}'.format(epoch, train_loss))

  model.eval()
  losses = 0
  tar_list = []
  pred_list = []

  with torch.no_grad():
    with tqdm(val_loader, unit="batch") as tepoch:
      for data, target in tepoch:
        pred = model(data.to(device))

        _, predicted = torch.max(pred.data, 1)
        tar_list.append(target.numpy())
        pred_list.append(predicted.cpu().numpy())        

        loss = loss_fn(pred, target.to(device))
        losses += loss.item()

  accuracy =  accuracy_score(np.concatenate(pred_list), np.concatenate(tar_list))
  val_loss = losses/len(val_loader)
  print('Train Epoch {} Training Loss {} and Accuracy {}'.format(epoch, val_loss, accuracy))

  if accuracy > val_acc:
    ckpt = {
        'model': model.state_dict(),
        'val_loss' : val_loss,
        'acc': accuracy,

    }
    torch.save(ckpt, 'best_model.pt')
    val_acc = accuracy

100%|██████████| 969/969 [02:15<00:00,  7.16batch/s]


Train Epoch 0 Training Loss 1.6678586120575942


100%|██████████| 243/243 [00:30<00:00,  7.88batch/s]


Train Epoch 0 Training Loss 0.843200599951018 and Accuracy 0.8886593678991944


100%|██████████| 969/969 [02:03<00:00,  7.85batch/s]


Train Epoch 1 Training Loss 0.7379004579764032


100%|██████████| 243/243 [00:29<00:00,  8.20batch/s]


Train Epoch 1 Training Loss 0.588487767143014 and Accuracy 0.934517661640157


In [11]:
print("https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html")
print("https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html")
print("https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html")

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
